In [3]:
# !pip install scikit-learn

In [45]:
import os
import pymysql
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import StandardScaler
import json


In [46]:
db = pymysql.connect(
    # host = "aws db 엔드포인트",
    host="",
    port=3306,
    user='admin',
    # passwd='관리자 계정 비밀번호',
    passwd="",
    db='movie_db',
    charset='utf8'
)

In [49]:
sql = "select * from movie"

movie_df = pd.read_sql(sql, db)
# movie_df 
## -> synopsis_vector 컬럼에 값이 추가된 상황

C:\Users\Playdata\AppData\Local\Temp\ipykernel_3728\3393629625.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  movie_df = pd.read_sql(sql, db)


In [50]:
# 범죄도시3 wordvector를 문자열로 변환한 synopsis_vector 컬럼값을 string_array에 할당
string_array = movie_df.loc[0,"synopsis_vector"]
string_array

'cv?=Y?????E=?U\x00>?U??t?\x0e?\x1f?ֻ?????a\t???%<?`??F?G??[??\x10??<PW*>\x1c?=>??s???{?\x02?^=?\x1ca???4=?[\x0e>f:?=??B?36x<u??<??~=\x00?N>??X?Ckc???y>Mm??].5>L\x16??zz?=?-T>@H?>??x??$/??rؼ?\x0b?9b??=??f?A\x0fB<?\x1f~>\x1ce]??p\x0e?L;h>\x0f??=???=?I?=?\x08?=S?\x04>y?ӽwO?=??:=?\x08\x02>??\x18??u??\x1e?f??6\x19???`??r\x19?\x1dc?<?2??>?3???轙?a>??;>\x17j?>#\t\x1f=?b\t?\x06ힽ`\x16?=??輦??=??2=R\x16??\x17t_?Y\x1b|?'

In [51]:
# 다시 numpy 배열로 반환
numpy_array = np.fromstring(string_array, dtype="float32")
# numpy_array

C:\Users\Playdata\AppData\Local\Temp\ipykernel_3728\714448942.py:2: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  numpy_array = np.fromstring(string_array, dtype="float32")


In [20]:
# 모든 행의 synopsis_vector 컬럼 값 출력
# movie_df.loc[:,"synopsis_vector"]


In [52]:

# 모든 행의 synopsis_vector 컬럼 값을 numpy 배열로 반환
movie_df.loc[:,"synopsis_vector_numpy"] = movie_df.loc[:,"synopsis_vector"].apply(lambda x:np.fromstring(x,dtype="float32"))

C:\Users\Playdata\AppData\Local\Temp\ipykernel_3728\1359174774.py:2: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  movie_df.loc[:,"synopsis_vector_numpy"] = movie_df.loc[:,"synopsis_vector"].apply(lambda x:np.fromstring(x,dtype="float32"))


In [25]:
# movie_df.loc[:,"synopsis_vector_numpy"]
# movie_df

In [53]:
# synopsis_vector_numpy 컬럼을 리스트로 변환
movie_df["synopsis_vector_numpy"].tolist()

## 이를 2차원 배열로 변경
np.array(movie_df["synopsis_vector_numpy"].tolist())

array([[0.04674376, 0.74706036, 0.04815602, ..., 0.74643433, 0.87286514,
        0.9847923 ],
       [0.0466605 , 0.74850225, 0.04522612, ..., 0.74649423, 0.8564338 ,
        0.9814338 ],
       [0.04672932, 0.74698406, 0.04254937, ..., 0.9111207 , 0.8524217 ,
        0.8876838 ],
       ...,
       [0.09220782, 0.74652755, 0.04989849, ..., 0.74705774, 0.9501855 ,
        0.7470588 ],
       [0.04666651, 0.747061  , 0.04717946, ..., 0.74621993, 0.86034006,
        0.96951663],
       [0.04674737, 0.74611276, 0.04107594, ..., 0.89252084, 0.7711208 ,
        0.85683155]], dtype=float32)

#### 데이터의 각 `column` 의 평균을 뺀 다음 표준편차로 나눠 평균을 0으로 표준편차를 1로 변환하는 `StandardScalar` 객체 생성

In [54]:
scaler = StandardScaler()

In [55]:
scaler.fit(np.array(movie_df["synopsis_vector_numpy"].tolist()))

StandardScaler()

In [56]:
# scaler.mean_, scaler.var_

In [57]:
scaler.transform(np.array(movie_df["synopsis_vector_numpy"].tolist()))

array([[-0.30696577,  0.52612716, -0.08184822, ..., -0.2642401 ,
         0.19448471,  0.8089791 ],
       [-0.31520057,  0.5292704 , -0.60991794, ..., -0.26399246,
         0.03366043,  0.7919346 ],
       [-0.3083938 ,  0.52596086, -1.0923604 , ...,  0.4165603 ,
        -0.00560872,  0.31614688],
       ...,
       [ 4.18943   ,  0.5249657 ,  0.23220453, ..., -0.261663  ,
         0.9512699 , -0.39753476],
       [-0.31460592,  0.5261286 , -0.25785848, ..., -0.2651264 ,
         0.07189348,  0.7314541 ],
       [-0.30660912,  0.5240615 , -1.3579227 , ...,  0.3396699 ,
        -0.8013544 ,  0.15956952]], dtype=float32)

In [58]:
movie_df["synopsis_vector_numpy_scale"] = scaler.transform(np.array(movie_df["synopsis_vector_numpy"].tolist())).tolist()

In [36]:
# movie_df["synopsis_vector_numpy_scale"]
# movie_df

In [59]:

# synopsis_vector_numpy_scale 컬럼의 유클리드 거리를 계산
sim_score = euclidean_distances(movie_df["synopsis_vector_numpy_scale"].tolist(), movie_df["synopsis_vector_numpy_scale"].tolist())

sim_score

array([[8.42936970e-08, 5.13615301e+00, 8.64671741e+00, ...,
        9.53234802e+00, 6.34011023e+00, 9.88013714e+00],
       [5.13615301e+00, 1.19209290e-07, 9.32985821e+00, ...,
        9.43824077e+00, 5.85092072e+00, 1.01179555e+01],
       [8.64671741e+00, 9.32985821e+00, 2.38418579e-07, ...,
        1.17890169e+01, 7.72812140e+00, 8.54027247e+00],
       ...,
       [9.53234802e+00, 9.43824077e+00, 1.17890169e+01, ...,
        3.95372485e-07, 8.69157840e+00, 1.26814832e+01],
       [6.34011023e+00, 5.85092072e+00, 7.72812140e+00, ...,
        8.69157840e+00, 1.19209290e-07, 9.31069456e+00],
       [9.88013714e+00, 1.01179555e+01, 8.54027247e+00, ...,
        1.26814832e+01, 9.31069456e+00, 0.00000000e+00]])

In [69]:
# sim_score[0]

sim_df = pd.DataFrame(data=sim_score)
sim_df

,0,1,2,3,4,5,6,7,8,9,...,660,661,662,663,664,665,666,667,668,669
0,8.429370e-08,5.136153e+00,8.646717e+00,12.717427,1.018700e+01,12.403767,11.143185,10.126189,9.589855,12.566201,...,9.572444,7.920480,7.967869,8.999684,11.312161,1.829076e+01,1.214209e+01,9.532348e+00,6.340110e+00,9.880137
1,5.136153e+00,1.192093e-07,9.329858e+00,13.309175,1.021038e+01,12.633945,10.280917,9.709438,9.302442,12.537936,...,9.698906,8.290173,8.825379,8.711023,11.996958,1.803300e+01,1.212900e+01,9.438241e+00,5.850921e+00,10.117956
2,8.646717e+00,9.329858e+00,2.384186e-07,11.677614,1.299230e+01,10.235586,13.140918,11.922495,11.803037,9.675180,...,10.443660,8.268904,10.143057,10.538910,8.521542,1.998530e+01,1.378252e+01,1.178902e+01,7.728121e+00,8.540272
3,1.271743e+01,1.330918e+01,1.167761e+01,0.000000,1.627476e+01,9.814536,16.121167,15.119422,15.448566,10.596105,...,14.288903,12.143743,13.089000,12.968475,10.624838,2.128302e+01,1.595866e+01,1.486802e+01,1.261108e+01,8.930263
4,1.018700e+01,1.021038e+01,1.299230e+01,16.274764,2.384186e-07,15.418483,9.197911,11.252566,7.571483,15.408927,...,9.822064,12.418254,11.888827,11.200681,15.082083,1.630181e+01,1.012789e+01,1.123512e+01,1.024200e+01,13.554413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665,1.829076e+01,1.803300e+01,1.998530e+01,21.283023,1.630181e+01,21.082383,17.506642,19.553486,17.490387,21.808727,...,18.638600,19.575786,19.546641,19.561725,19.275827,3.371748e-07,1.821044e+01,1.884444e+01,1.809343e+01,20.243250
666,1.214209e+01,1.212900e+01,1.378252e+01,15.958660,1.012789e+01,16.365299,10.729235,11.680033,10.480666,15.971714,...,9.814171,12.125656,12.489581,12.167447,16.119065,1.821044e+01,1.685874e-07,1.217558e+01,1.148830e+01,14.230093
667,9.532348e+00,9.438241e+00,1.178902e+01,14.868024,1.123512e+01,12.614945,11.378572,9.178670,9.351144,13.176898,...,9.725712,9.591533,11.403170,10.083518,14.063271,1.884444e+01,1.217558e+01,3.953725e-07,8.691578e+00,12.681483
668,6.340110e+00,5.850921e+00,7.728121e+00,12.611081,1.024200e+01,11.947137,10.672641,9.363375,8.500983,11.236314,...,9.374325,7.782079,8.933411,7.709240,10.965142,1.809343e+01,1.148830e+01,8.691578e+00,1.192093e-07,9.310695


In [71]:
sim_df.index = movie_df["title"]
sim_df.columns = movie_df["title"]

# sim_df

In [75]:
# 영화 존 윅 - 리로드 와 거리가 가까운 순으로 TOP 5 조회
## 자기 자신 제외

sim_df["존 윅 - 리로드"].sort_values()[1:6]

title
보이스                   5.091718
스파이더맨: 어크로스 더 유니버스    5.548917
언어의 정원                5.808760
뽀로로 극장판 드래곤캐슬 대모험     5.847725
라스트 필름                5.884819
Name: 존 윅 - 리로드, dtype: float64

In [76]:
pd.DataFrame(sim_df["존 윅 - 리로드"].sort_values()[1:6]).reset_index()

,title,존 윅 - 리로드
0,보이스,5.091718
1,스파이더맨: 어크로스 더 유니버스,5.548917
2,언어의 정원,5.808760
3,뽀로로 극장판 드래곤캐슬 대모험,5.847725
4,라스트 필름,5.884819


In [77]:
sim_df["존 윅 - 리로드"].sort_values()[1:6].reset_index().values.tolist()

[['보이스', 5.091718251403315],
 ['스파이더맨: 어크로스 더 유니버스', 5.5489172268224145],
 ['언어의 정원', 5.8087597936066215],
 ['뽀로로 극장판 드래곤캐슬 대모험', 5.847724703868203],
 ['라스트 필름', 5.884819403316852]]

In [78]:
# 위의 영화 존 윅 - 리로드 와 거리가 가까운 순으로 자기 자신을 제외한 TOP 5 값을
## DataFrame 으로 바꾸고 인덱스를 지우고 해당 영화 제목을 넣은 값에서
### numpy 배열 -> 리스트로 바꾼 후 다시 DataFrame에 담아 json 문자열로 변환한 결과

result = pd.DataFrame(sim_df["존 윅 - 리로드"].sort_values()[1:6]).reset_index().values.tolist()

json.dumps(result, ensure_ascii=False)

'[["보이스", 5.091718251403315], ["스파이더맨: 어크로스 더 유니버스", 5.5489172268224145], ["언어의 정원", 5.8087597936066215], ["뽀로로 극장판 드래곤캐슬 대모험", 5.847724703868203], ["라스트 필름", 5.884819403316852]]'

In [ ]:
db.close()